### Import necessary libraries, set options

In [ ]:
import numpy as np
import os
import pandas as pd
import warnings

pd.set_option('display.max_columns', 125)
warnings.filterwarnings("ignore")

### Between-individual

In [ ]:
path_to_data = os.path.join("path/to/data/processed-data")

data_final = pd.read_pickle(path_to_data + "/pkl/data_final.pkl")
print(len(data_final))
data_final.head()

In [ ]:
data_final_coop = data_final.drop_duplicates(subset = ["session_no", 
                                                       "pid", 
                                                       "version",
                                                       "round"], keep = "first").reset_index(drop = True)

data_final_coop.to_pickle(path_to_data + "/pkl/data_final_coop.pkl")
data_final_coop.to_csv(path_to_data + "/csv/data_final_coop.csv")

print(len(data_final))
print(len(data_final_coop))

data_final_coop.head()

In [ ]:
data_pid_version = data_final.drop_duplicates(subset = ["pid", "version"], keep = "first").reset_index(drop = True)

data_pid_version.to_pickle(path_to_data + "/pkl/data_pid_version.pkl")
data_pid_version.to_csv(path_to_data + "/csv/data_pid_version.csv")

print(len(data_final))
print(len(data_pid_version))

data_pid_version.head()

In [ ]:
data_pid = data_final.drop_duplicates(subset = "pid", keep = "first").reset_index(drop = True)

data_pid.to_pickle(path_to_data + "/pkl/data_pid.pkl")
data_pid.to_csv(path_to_data + "/csv/data_pid.csv")

print(len(data_final))
print(len(data_pid))

data_pid.head()

### Between-session

In [ ]:
session_data = data_final.groupby(['session_no', 
                                   'version', 
                                   'round']).mean()[['earned', 'equal',
                                                     'addChoice', 'cutChoice',
                                                     'num_players',
                                                     'coopChoiceAlter_add',
                                                     'coopChoiceAlter_cut',
                                                     'giniLocal']].reset_index()

session_data.to_pickle(path_to_data + "/pkl/session_data.pkl")
session_data.to_csv(path_to_data + "/csv/session_data.csv")

print(len(session_data))

session_data.head()

In [ ]:
session_data_coop = data_final_coop.groupby(['session_no', 
                                             'version', 
                                             'round']).mean()[['earned', 'equal', 
                                                               'coopChoice',
                                                               'num_players',
                                                               'scoreBeforeCoop',
                                                               'giniLocal']].reset_index()

session_data_coop.to_pickle(path_to_data + "/pkl/session_data_coop.pkl")
session_data_coop.to_csv(path_to_data + "/csv/session_data_coop.csv")

print(len(session_data_coop))

session_data_coop.head()

In [ ]:
session_data_pid_version = data_pid_version.groupby(['session_no', 
                                                     'version']).mean()[['earned', 'equal', 
                                                                         'f_score', 'trust_score',
                                                                         'num_players',
                                                                         'change_in_score']].reset_index()

session_data_pid_version.to_pickle(path_to_data + "/pkl/session_data_pid_version.pkl")
session_data_pid_version.to_csv(path_to_data + "/csv/session_data_pid_version.csv")

print(len(session_data_pid_version))

session_data_pid_version.head()

### Within-individual

In [ ]:
vars_to_join_on = ["session_no", "pid"]

In [ ]:
player_vars = [
    "score1",
    "more_fair", "prefer",
    "why_coop_list", "why_coop_other",
    "num_other", "age", "gender", "race", "education", "income", "politics",
    "country", "ip_to_country"
]

In [ ]:
data_within_player = data_final[vars_to_join_on + player_vars]
data_within_player = data_within_player.drop_duplicates(subset = "pid", keep = "first").reset_index(drop = True)

print(len(data_within_player))
data_within_player.head()

In [ ]:
round_vars = [
    "round",
    "version",
    "earned", "equal", "condition",
    "coopChoice", "scoreBeforeCoop",
    "addChoice", "scoreEgo_add", "scoreAlter_add", "coopChoiceEgo_add", "coopChoiceAlter_add",
    "cutChoice", "scoreEgo_cut", "scoreAlter_cut", "coopChoiceEgo_cut", "coopChoiceAlter_cut",
    "avgCoopGlobal", "avgCoopLocal",
    "giniGlobal", "giniLocal",
    "change_in_score",
    "f_score",
    "trust_score",
    "neighborsList",
    "num_players"
]

v1_vars = ["earned1", "equal1", "first_version",  "score2", "f_score2", "trust_score2"]
v2_vars = ["earned2", "equal2", "second_version", "score3", "f_score3", "trust_score3"]

In [ ]:
data_within_v1 = data_final[vars_to_join_on + round_vars + v1_vars]
data_within_v1 = data_within_v1[data_within_v1['version'] == 1].reset_index(drop = True)
print(len(data_within_v1))
data_within_v1.head()

In [ ]:
data_within_v2 = data_final[vars_to_join_on + round_vars + v2_vars]
data_within_v2 = data_within_v2[data_within_v2['version'] == 2].reset_index(drop = True)
print(len(data_within_v2))
data_within_v2.head()

In [ ]:
data_within_round = data_within_v1.merge(right = data_within_v2, 
                                         how = "inner",
                                         on = ["session_no", "pid", "round"], 
                                         suffixes = ("_v1", "_v2"))

print(len(data_within_round))
data_within_round.head()

In [ ]:
data_within_round = data_within_round.merge(right = data_within_player,
                                            how = "inner",
                                            on = ["session_no", "pid"])

print(len(data_within_round))
data_within_round.head()

In [ ]:
data_within = data_within_round[["session_no", "pid", "round", 
                                 "earned_v1", "equal_v1",
                                 "earned_v2", "equal_v2"] + player_vars]
data_within.head()

In [ ]:
data_within["delta_earned"] = data_within_round["earned_v2"] - data_within_round["earned_v1"]
data_within["delta_equal"]  = data_within_round["equal_v2"]  - data_within_round["equal_v1"]

In [ ]:
data_within["delta_coopChoice"]      = data_within_round["coopChoice_v2"]      - \
                                       data_within_round["coopChoice_v1"]
data_within["delta_scoreBeforeCoop"] = data_within_round["scoreBeforeCoop_v2"] - \
                                       data_within_round["scoreBeforeCoop_v1"]

In [ ]:
data_within["delta_addChoice"]           = data_within_round["addChoice_v2"]           - \
                                           data_within_round["addChoice_v1"]
data_within["delta_scoreEgo_add"]        = data_within_round["scoreEgo_add_v2"]        - \
                                           data_within_round["scoreEgo_add_v1"]
data_within["delta_scoreAlter_add"]      = data_within_round["scoreAlter_add_v2"]      - \
                                           data_within_round["scoreAlter_add_v1"]
data_within["delta_coopChoiceEgo_add"]   = data_within_round["coopChoiceEgo_add_v2"]   - \
                                           data_within_round["coopChoiceEgo_add_v1"]
data_within["delta_coopChoiceAlter_add"] = data_within_round["coopChoiceAlter_add_v2"] - \
                                           data_within_round["coopChoiceAlter_add_v1"]

In [ ]:
data_within["delta_cutChoice"]           = data_within_round["cutChoice_v2"]           - \
                                           data_within_round["cutChoice_v1"]
data_within["delta_scoreEgo_cut"]        = data_within_round["scoreEgo_cut_v2"]        - \
                                           data_within_round["scoreEgo_cut_v1"]
data_within["delta_scoreAlter_cut"]      = data_within_round["scoreAlter_cut_v2"]      - \
                                           data_within_round["scoreAlter_cut_v1"]
data_within["delta_coopChoiceEgo_cut"]   = data_within_round["coopChoiceEgo_cut_v2"]   - \
                                           data_within_round["coopChoiceEgo_cut_v1"]
data_within["delta_coopChoiceAlter_cut"] = data_within_round["coopChoiceAlter_cut_v2"] - \
                                           data_within_round["coopChoiceAlter_cut_v1"]

In [ ]:
data_within["delta_avgCoopGlobal"] = data_within_round["avgCoopGlobal_v2"] - \
                                     data_within_round["avgCoopGlobal_v1"]
data_within["delta_avgCoopLocal"]  = data_within_round["avgCoopLocal_v2"]  - \
                                     data_within_round["avgCoopLocal_v1"]

In [ ]:
data_within["delta_giniGlobal"] = data_within_round["giniGlobal_v2"] - data_within_round["giniGlobal_v1"]
data_within["delta_giniLocal"]  = data_within_round["giniLocal_v2"]  - data_within_round["giniLocal_v1"]

In [ ]:
data_within["delta_change_in_score"] = data_within_round["change_in_score_v2"] - \
                                       data_within_round["change_in_score_v1"]

In [ ]:
data_within["delta_f_score"] = data_within_round["f_score_v2"] - data_within_round["f_score_v1"]

In [ ]:
data_within["delta_trust_score"] = data_within_round["trust_score_v2"] - data_within_round["trust_score_v1"]

In [ ]:
data_within["delta_game_score"] = data_within_round["score3"] - data_within_round["score2"]

In [ ]:
def count_num_neighbors(row):
    if (row["neighborsList_v2"] == row["neighborsList_v2"]) and \
       (row["neighborsList_v1"] == row["neighborsList_v1"]):
        return len(row["neighborsList_v2"]) - len(row["neighborsList_v1"])
    else:
        return np.nan

data_within["delta_num_neighbors"] = data_within_round.apply(count_num_neighbors, axis = 1)

In [ ]:
data_within["delta_num_players"] = data_within_round["num_players_v2"] - data_within_round["num_players_v1"]

In [ ]:
print(len(data_within))
data_within.head()

In [ ]:
data_within.to_pickle(path_to_data + "/pkl/data_within.pkl")
data_within.to_csv(path_to_data + "/csv/data_within.csv")

In [ ]:
data_within_coop = data_within.drop_duplicates(subset = ["session_no", "pid", "round"], keep = "first").reset_index()
data_within_coop.to_pickle(path_to_data + "/pkl/data_within_coop.pkl")
data_within_coop.to_csv(path_to_data + "/csv/data_within_coop.csv")

In [ ]:
data_within_pid = data_within.drop_duplicates(subset = ["session_no", "pid"], keep = "first").reset_index()
data_within_pid.to_pickle(path_to_data + "/pkl/data_within_pid.pkl")
data_within_pid.to_csv(path_to_data + "/csv/data_within_pid.csv")

### Within-session

In [ ]:
session_data_v1 = data_final[data_final['version'] == 1].groupby(
    ['session_no', 'round']).mean()[['version', 'earned', 'equal', 'addChoice', 'cutChoice',
                                     'coopChoiceAlter_add', 'coopChoiceAlter_cut', 
                                     'giniLocal']].reset_index()

session_data_v1.columns = ["session_no", "round", "version1", "earned1", "equal1", "addChoice1", "cutChoice1",
                           "coopChoiceAlter_add1", "coopChoiceAlter_cut1", "giniLocal1"]

session_data_v2 = data_final[data_final['version'] == 2].groupby(
    ['session_no', 'round']).mean()[['version', 'earned', 'equal', 'addChoice', 'cutChoice',
                                     'coopChoiceAlter_add', 'coopChoiceAlter_cut',
                                     'giniLocal']].reset_index()

session_data_v2.columns = ["session_no", "round", "version2", "earned2", "equal2", "addChoice2", "cutChoice2",
                           "coopChoiceAlter_add2", "coopChoiceAlter_cut2", "giniLocal2"]

session_data_within = session_data_v1.merge(right = session_data_v2, 
                                            how = "inner",
                                            on = ["session_no", "round"])

session_data_within["delta_earned"]    = session_data_within["earned2"]    - session_data_within["earned1"]
session_data_within["delta_equal"]     = session_data_within["equal2"]     - session_data_within["equal1"]
session_data_within["delta_addChoice"] = session_data_within["addChoice2"] - session_data_within["addChoice1"]
session_data_within["delta_cutChoice"] = session_data_within["cutChoice2"] - session_data_within["cutChoice1"]

session_data_within["delta_coopChoiceAlter_add"] = session_data_within["coopChoiceAlter_add2"] - \
                                                   session_data_within["coopChoiceAlter_add1"]
session_data_within["delta_coopChoiceAlter_cut"] = session_data_within["coopChoiceAlter_cut2"] - \
                                                   session_data_within["coopChoiceAlter_cut1"]

session_data_within["delta_giniLocal"] = session_data_within["giniLocal2"] - session_data_within["giniLocal1"]

session_data_within = session_data_within[["session_no", 
                                           "round", 
                                           "delta_earned", 
                                           "delta_equal", 
                                           "earned1", "equal1", "earned2", "equal2", 
                                           "delta_addChoice",
                                           "delta_cutChoice",
                                           "delta_coopChoiceAlter_add",
                                           "delta_coopChoiceAlter_cut",
                                           "delta_giniLocal"]]

session_data_within.to_pickle(path_to_data + "/pkl/session_data_within.pkl")
session_data_within.to_csv(path_to_data + "/csv/session_data_within.csv")

In [ ]:
session_data_v1 = data_final_coop[data_final_coop['version'] == 1].groupby(
    ['session_no', 'round']).mean()[['version', 'earned', 'equal', 'coopChoice', 'scoreBeforeCoop',
                                     'giniLocal']].reset_index()

session_data_v1.columns = ["session_no", "round", "version1", "earned1", "equal1", "coopChoice1", "scoreBeforeCoop1",
                           "giniLocal1"]

session_data_v2 = data_final_coop[data_final_coop['version'] == 2].groupby(
    ['session_no', 'round']).mean()[['version', 'earned', 'equal', 'coopChoice', 'scoreBeforeCoop',
                                     'giniLocal']].reset_index()

session_data_v2.columns = ["session_no", "round", "version2", "earned2", "equal2", "coopChoice2", "scoreBeforeCoop2",
                           "giniLocal2"]

session_data_within = session_data_v1.merge(right = session_data_v2, 
                                            how = "inner",
                                            on = ["session_no", "round"])

session_data_within["delta_earned"]     = session_data_within["earned2"]     - session_data_within["earned1"]
session_data_within["delta_equal"]      = session_data_within["equal2"]      - session_data_within["equal1"]
session_data_within["delta_coopChoice"] = session_data_within["coopChoice2"] - session_data_within["coopChoice1"]

session_data_within["delta_scoreBeforeCoop"] = session_data_within["scoreBeforeCoop2"] - \
                                               session_data_within["scoreBeforeCoop1"]

session_data_within["delta_giniLocal"] = session_data_within["giniLocal2"] - session_data_within["giniLocal1"]

session_data_within = session_data_within[["session_no", 
                                           "round", 
                                           "delta_earned", 
                                           "delta_equal", 
                                           "earned1", "equal1", "earned2", "equal2", 
                                           "delta_coopChoice",
                                           "delta_scoreBeforeCoop",
                                           "delta_giniLocal"]]

session_data_within.to_pickle(path_to_data + "/pkl/session_data_within_coop.pkl")
session_data_within.to_csv(path_to_data + "/csv/session_data_within_coop.csv")

In [ ]:
session_data_v1 = data_pid_version[data_pid_version['version'] == 1].groupby(
    ['session_no']).mean()[['version', 'earned', 'equal', 'f_score', 'trust_score', 'change_in_score']].reset_index()

session_data_v1.columns = ["session_no", "version1", "earned1", "equal1", "f_score1", "trust_score1",
                           "change_in_score1"]

session_data_v2 = data_pid_version[data_pid_version['version'] == 2].groupby(
    ['session_no']).mean()[['version', 'earned', 'equal', 'f_score', 'trust_score', 'change_in_score']].reset_index()

session_data_v2.columns = ["session_no", "version2", "earned2", "equal2", "f_score2", "trust_score2",
                           "change_in_score2"]

session_data_within = session_data_v1.merge(right = session_data_v2, 
                                            how = "inner",
                                            on = ["session_no"])

session_data_within["delta_earned"]      = session_data_within["earned2"]      - session_data_within["earned1"]
session_data_within["delta_equal"]       = session_data_within["equal2"]       - session_data_within["equal1"]
session_data_within["delta_f_score"]     = session_data_within["f_score2"]     - session_data_within["f_score1"]
session_data_within["delta_trust_score"] = session_data_within["trust_score2"] - session_data_within["trust_score1"]

session_data_within["delta_change_in_score"] = session_data_within["change_in_score2"] - \
                                               session_data_within["change_in_score1"]

session_data_within = session_data_within[["session_no", 
                                           "delta_earned", 
                                           "delta_equal", 
                                           "earned1", "equal1", "earned2", "equal2", 
                                           "delta_f_score",
                                           "delta_trust_score",
                                           "delta_change_in_score"]]

session_data_within.to_pickle(path_to_data + "/pkl/session_data_within_pid.pkl")
session_data_within.to_csv(path_to_data + "/csv/session_data_within_pid.csv")